# SYMBOLIC STUDY

## **SYSTEM DESCRIPTION**
-------------------------

In [ ]:
from IPython.display import display, Markdown
display(Markdown(filename='README.md'))

---------------------------------------------------------------
---------------------------------------------------------------

## **SYMBOLIC TOPOLOGY**
------------------------

In this section, we create the symbolic topology that captures the topological layout that we just discussed.</br>
Defining the topology is very simple. We start by importing the ```template_topology``` class and create a new instance that represents our symbolic model. Then we start adding the components we discussed earlier, starting by the bodies, then the joints, actuators and forces, and thats it.</br>
These components will be represented symbolically, and therefore there is no need for any numerical inputs at this step.

The system is stored in a form of a network graph that stores all the data needed for the assemblage of the system equations later. But even before the assemblage process, we can gain helpful insights about our system as well be shown.


In [2]:
import os
from uraeus.smbd.systems import template_topology, configuration

In [3]:
# Getting the database root directory which is three levels up from here
database_directory = os.path.abspath('../../../')

# Creating "data" directory to store generated system data
os.makedirs('data', exist_ok=True)

In [4]:
model_name = 'chassis'

In [5]:
sym_model = template_topology(model_name)

### Adding Bodies

In [6]:
sym_model.add_body('chassis')

### Adding Joints

In [7]:
# Adding Joints
# =============

### Adding Actuators

In [8]:
# Adding Actuators
# ================

### Adding Forces

In [9]:
# Adding Forces
# =============
sym_model.add_force.generic_load('aero_drag', 'rbs_chassis')

### **ASSEMBLING**

This is the last step of the symbolic building process, where we make the system starts the assemblage process of the governing equations, which will be used then in the code generation for the numerical simulation, and also can be used for further symbolic manipulations.</br>

*Note: The equations' notations will be discussed in the tool documentation files.*

In [10]:
sym_model.save('data')
sym_model.assemble()

---------------------------------------------------------------
---------------------------------------------------------------

## **SYSTEM CONFIGURATION**
---------------------------

In this step we define a symbolic configuration of our symbolic topology. As you may have noticed in the symbolic topology building step, we only cared about the *'topology'*, thats is the system bodies and their connectivity, and we did not care explicitly with how these components are configured in space.</br>
In order to create a valid numerical simulation session, we have to provide the system with its numerical configuration needed, for example, the joints' locations and orientations. The symbolic topology in its raw form will require you to manually enter all these numerical arguments, which can be cumbersome even for smaller systems. This can be checked by checking the configuration inputs of the symbolic configuration as ```sym_config.config.input_nodes```

Here we start by stating the symbolic inputs we wish to use instead of the default inputs set, and then we define the relation between these newly defined arguments and the original ones. The details of this process will be provided in the documentation.

In [11]:
config_name = '%s_cfg'%model_name
sym_config = configuration(config_name, sym_model)

### CONFIGURATION INPUTS

In [12]:
# Adding UserInputs
# =================
sym_config.add_point.UserInput('CG')

### CONFIGURATION RELATIONS

In [13]:
# Aero Drag Force:
# ====================
sym_config.add_relation.Equal_to('pt1_fas_aero_drag', ('hps_CG',))

### GEOMETRIES

Here we start defining basic geometric shapes that can represents the shapes of the bodies in our system. This serves two points:
- Visualization and Animation in blender.
- Evaluating the bodies inertia properties from these basic geometries instead of explicit definition.

In [14]:
# Creating Geometries
# ===================
sym_config.add_scalar.UserInput('CG_radius')

sym_config.add_geometry.Sphere_Geometry('CG', ('hps_CG','s_CG_radius'))
sym_config.assign_geometry_to_body('rbs_chassis', 'gms_CG', False)
sym_config.add_relation.Equal_to('R_rbs_chassis', ('gms_CG.R',))
sym_config.add_relation.Equal_to('P_rbs_chassis', ('gms_CG.P',))

### ASSEMBLING

In [15]:
sym_config.export_JSON_file('data')

## CODE GENERATION

### Python Numerical Environment

In [16]:
from uraeus.nmbd.python import templatebased_project

In [17]:
project = templatebased_project(database_directory)
project.create_dirs()

In [18]:
project.write_topology_code(sym_model)

File full path : C:\khaledghobashy\github\uraeus_fsae\numenv\python\templates\chassis.py
